In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [3]:
# 데이터 로드
data = pd.read_csv("C:\\Users\\LG\\Downloads\\diabetes.csv")

In [6]:
# Outcome 제거, BMI를 예측 타겟으로 설정
X = data.drop(columns=['Outcome', 'BMI']).values   # BMI와 Outcome 제외 → 나머지 7개 특성
y = data['BMI'].values.astype(np.float32)          # 예측 타겟: BMI (회귀)

In [8]:
# 정규화
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [10]:
# 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# 텐서로 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [14]:
# DataLoader 구성
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [16]:
# MLP 회귀 모델 정의
class BMIRegressionModel(nn.Module):
    def __init__(self):
        super(BMIRegressionModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(7, 64),     # 입력 차원: 7
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)      # 출력: BMI (회귀)
        )

    def forward(self, x):
        return self.model(x)

In [18]:
# 학습 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BMIRegressionModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [20]:
# 학습 루프
model.train()
for epoch in range(50):
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

Epoch 1, Loss: 1041.8297
Epoch 2, Loss: 1010.5800
Epoch 3, Loss: 889.6990
Epoch 4, Loss: 726.7693
Epoch 5, Loss: 517.2225
Epoch 6, Loss: 313.6847
Epoch 7, Loss: 162.8200
Epoch 8, Loss: 98.7786
Epoch 9, Loss: 85.1188
Epoch 10, Loss: 77.5389
Epoch 11, Loss: 75.9847
Epoch 12, Loss: 71.7787
Epoch 13, Loss: 68.2477
Epoch 14, Loss: 66.8928
Epoch 15, Loss: 66.0953
Epoch 16, Loss: 63.7183
Epoch 17, Loss: 61.5986
Epoch 18, Loss: 61.3456
Epoch 19, Loss: 58.4315
Epoch 20, Loss: 58.2348
Epoch 21, Loss: 59.1689
Epoch 22, Loss: 56.3399
Epoch 23, Loss: 55.6525
Epoch 24, Loss: 59.4944
Epoch 25, Loss: 62.1947
Epoch 26, Loss: 53.2449
Epoch 27, Loss: 51.2777
Epoch 28, Loss: 52.3383
Epoch 29, Loss: 52.3614
Epoch 30, Loss: 49.0260
Epoch 31, Loss: 49.9240
Epoch 32, Loss: 53.4309
Epoch 33, Loss: 49.4434
Epoch 34, Loss: 46.9167
Epoch 35, Loss: 48.9908
Epoch 36, Loss: 46.5292
Epoch 37, Loss: 45.4281
Epoch 38, Loss: 45.1257
Epoch 39, Loss: 46.4136
Epoch 40, Loss: 46.0067
Epoch 41, Loss: 44.0930
Epoch 42, Loss: 

In [22]:
# 평가
model.eval()
preds, actuals = [], []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        output = model(X_batch).cpu().numpy()
        preds.extend(output)
        actuals.extend(y_batch.numpy())

mse = mean_squared_error(actuals, preds)
print(f"Test MSE: {mse:.4f}")

Test MSE: 68.8765
